# Задача 2.1
##### Обмен секретным ключом по алгоритму Диффи-Хеллмана с аутентификацией на основе ЭП Эль-Гамаля

В предыдущей задаче мы реализовали обмен ключами Диффи-Хеллмана, но не учли возможность MITM-атаки. Для предотвращения этой атаки добавим аутентификацию с помощью электронной подписи Эль-Гамаля.

### Этапы реализации:

1) **Выбор общих параметров:**

    Выберите большое простое число $p$ и генератор $g$ группы.

2) **Генерация ключей:**

    Алиса генерирует пару ключей Эль-Гамаля: секретный ключ $a_{priv}$ и открытый ключ $a_{pub}$.

    Боб генерирует пару ключей Эль-Гамаля: секретный ключ $b_{priv}$ и открытый ключ $b_{pub}$.

    Алиса выбирает случайное число $a$ (секретный ключ Диффи-Хеллмана) и вычисляет $A = g^a \mod p$ (открытый ключ Диффи-Хеллмана).

    Боб выбирает случайное число $b$ (секретный ключ Диффи-Хеллмана) и вычисляет $B = g^b \mod p$ (открытый ключ Диффи-Хеллмана).
   
3) **Обмен ключами и аутентификация:**

    Алиса отправляет Бобу свой открытый ключ Диффи-Хеллмана $A$ и свою подпись Эль-Гамаля для $A$, созданную с помощью $a_{priv}$.
    
    Боб проверяет подпись Алисы, используя $a_{pub}$. Если подпись верна, Боб отправляет Алисе свой открытый ключ Диффи-Хеллмана $B$ и свою подпись Эль-Гамаля для B, созданную с помощью $b_{priv}$.
    
    Алиса проверяет подпись Боба, используя $b_{pub}$.

4) **Вычисление общего секретного ключа:**

    Алиса вычисляет общий секретный ключ $K = B^a \mod p$.

    Боб вычисляет общий секретный ключ $K = A^b \mod p$.

In [1]:
import random
import math
from dataclasses import dataclass
from typing import Tuple, NamedTuple

@dataclass
class ElGamalSignature:
    r: int
    s: int

class KeyPair(NamedTuple):
    private: int
    public: int

class AuthenticatedDHParticipant:
    def __init__(self, p: int, g: int, name: str):
        self.p = p
        self.g = g
        self.name = name
        self._elgamal_keys = None
        self._dh_keys = None
        self._shared_key = None

    def generate_keys(self) -> Tuple[int, int]:
        priv_key = random.randint(2, self.p - 2)
        pub_key = pow(self.g, priv_key, self.p)
        self._elgamal_keys = KeyPair(priv_key, pub_key)

        dh_priv = random.randint(2, self.p - 2)
        dh_pub = pow(self.g, dh_priv, self.p)
        self._dh_keys = KeyPair(dh_priv, dh_pub)

        return self._dh_keys.public, self._elgamal_keys.public

    def sign_dh_key(self) -> ElGamalSignature:
        if not self._elgamal_keys or not self._dh_keys:
            raise ValueError("Ключи не сгенерированы")

        m = self._dh_keys.public
        
        while True:
            k = random.randint(2, self.p - 2)
            if math.gcd(k, self.p - 1) == 1:
                break

        r = pow(self.g, k, self.p)
        
        k_inv = pow(k, -1, self.p - 1)
        s = ((m - self._elgamal_keys.private * r) * k_inv) % (self.p - 1)
        
        if s == 0:
            return self.sign_dh_key()
            
        return ElGamalSignature(r, s)

    def verify_signature(self, message: int, signature: ElGamalSignature, public_key: int) -> bool:
        r, s = signature.r, signature.s
        
        if not (0 < r < self.p and 0 < s < self.p - 1):
            return False

        left_side = pow(self.g, message, self.p)
        right_side = (pow(public_key, r, self.p) * pow(r, s, self.p)) % self.p
        
        return left_side == right_side

    def compute_shared_key(self, other_public_key: int) -> int:
        if not self._dh_keys:
            raise ValueError("Ключи DH не сгенерированы")
        
        self._shared_key = pow(other_public_key, self._dh_keys.private, self.p)
        return self._shared_key

def run_authenticated_exchange() -> bool:
    p = 23
    g = 5
    
    print(f"Используются параметры: p = {p}, g = {g}")

    alice = AuthenticatedDHParticipant(p, g, "Alice")
    bob = AuthenticatedDHParticipant(p, g, "Bob")

    try:
        alice_dh_pub, alice_elgamal_pub = alice.generate_keys()
        print(f"Открытые ключи Алисы: DH = {alice_dh_pub}, ElGamal = {alice_elgamal_pub}")
        
        bob_dh_pub, bob_elgamal_pub = bob.generate_keys()
        print(f"Открытые ключи Боба: DH = {bob_dh_pub}, ElGamal = {bob_elgamal_pub}")

        alice_signature = alice.sign_dh_key()
        print(f"Подпись Алисы: r = {alice_signature.r}, s = {alice_signature.s}")
        
        if not bob.verify_signature(alice_dh_pub, alice_signature, alice_elgamal_pub):
            print("Ошибка проверки подписи Алисы")
            return False
        print("Подпись Алисы проверена успешно")

        bob_signature = bob.sign_dh_key()
        print(f"Подпись Боба: r = {bob_signature.r}, s = {bob_signature.s}")
        
        if not alice.verify_signature(bob_dh_pub, bob_signature, bob_elgamal_pub):
            print("Ошибка проверки подписи Боба")
            return False
        print("Подпись Боба проверена успешно")

        alice_shared = alice.compute_shared_key(bob_dh_pub)
        bob_shared = bob.compute_shared_key(alice_dh_pub)

        print(f"Общий ключ Алисы: {alice_shared}")
        print(f"Общий ключ Боба: {bob_shared}")

        return alice_shared == bob_shared

    except Exception as e:
        print(f"Ошибка при обмене ключами: {e}")
        return False

def main():
    if run_authenticated_exchange():
        print("Защищенный обмен ключами успешно завершен!")
    else:
        print("Ошибка при выполнении защищенного обмена ключами")

if __name__ == "__main__":
    main()

Используются параметры: p = 23, g = 5
Открытые ключи Алисы: DH = 8, ElGamal = 3
Открытые ключи Боба: DH = 22, ElGamal = 20
Подпись Алисы: r = 15, s = 20
Подпись Алисы проверена успешно
Подпись Боба: r = 21, s = 19
Подпись Боба проверена успешно
Общий ключ Алисы: 1
Общий ключ Боба: 1
Защищенный обмен ключами успешно завершен!
